# Lime kor
- 한국어 비속어 분포도 확인

In [ ]:
!pip install lime dataset

In [ ]:
from datasets import load_dataset
import pandas as pd

hate_dataset = load_dataset("SJ-Donald/kor-hate-sentence")
train2 = hate_dataset['train']
train1 = hate_dataset['validation']

train2 = pd.DataFrame(train2)
train1 = pd.DataFrame(train1)

train_all = pd.concat([train1, train2])
train_all = train_all.reset_index()

train_all = train_all[['문장', 'hate']]
train_all.columns=['comment_text', 'toxic']

# train2 = train2[['문장', 'hate']]
# train2.columns=['comment_text', 'toxic']


In [ ]:
x_train_text = train_all.comment_text
y_train = train_all.toxic

In [ ]:
print(x_train_text.head(), y_train.head())

In [ ]:
from matplotlib import pyplot as plt

plt.hist(y_train, bins=2)

In [ ]:
import sklearn, sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train_text)

In [ ]:
x_train[0]

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(x_train, y_train)

In [ ]:
pred = clf.predict(x_train)

print("정확도: ", sklearn.metrics.accuracy_score(y_train, pred))
print("confusion matrix: \n", sklearn.metrics.confusion_matrix(y_train, pred, normalize='pred'))

In [ ]:
def get_text_by_y_pred(y, p):
  return [i for i in range(x_train.shape[0]) if y_train[i]==y and pred[i]==p]

In [ ]:
tp = get_text_by_y_pred(0,0)
fn = get_text_by_y_pred(0,1)
fp = get_text_by_y_pred(1,0)
tn = get_text_by_y_pred(1,1)

In [ ]:
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer

def predict_pipe(x):
  x = vectorizer.transform(x)
  x = clf.predict(x)
  return x

# i 번째 훈련 데이터를 Lime으로 분석해서 노트북에 표시
def explain(i, text=None, cls=None):
  if not text and not cls:
    text, cls = x_train_text[i], y_train[i]
  class_names=["Normal", "Toxic"]

  # Lime 분석
  pipe = make_pipeline(vectorizer, clf)
  explainer = LimeTextExplainer(class_names=class_names)
  exp = explainer.explain_instance(text, pipe.predict_proba)

  # 분류기로 예측한 결과 표시
  pred = clf.predict(vectorizer.transform([text])[0])
  pred = "Toxic" if(pred == 1) else "Normal"
  cls = "Toxic" if(cls == 1) else "Normal"

  print()
  print(f"#{i} Predict: {pred} Real: {cls}")
  
  # 노트북에 표시
  exp.show_in_notebook(text=text)
  print()

In [ ]:
#for i in np.random.choice(fn, 10):
for i in [2312, 213, 123, 1424, 1234]:
  explain(i)

In [ ]:
text = """나는 너무 착하다.  mms://a352.l5671334351.c56713.n.lm.akamaistream.net/D/352/56713/v0001/reflector:34351"""
#eval(text, 1)
#explain(text, 1)
explain(1, text)

In [ ]:
# False Positive 로 분류된 값들의 문자열 길이 분포
fp_len = [min(len(x_train_text[i]), 500) for i in fp]
plt.hist(fp_len)

In [ ]:
import numpy as np

shorts = [i for i in fp if len(x_train_text[i]) < 100]
longs = [i for i in fp if len(x_train_text[i]) >= 100]

shorts = np.random.choice(shorts, 5)
longs = np.random.choice(longs, 5)

print(shorts)
for i in shorts:
  explain(i)

print(longs)
for i in longs:
  explain(i)

In [ ]:
# 문자열에서 악성 단어를 찾아내서 *로 바꿉니다.
def filter_toxic(text):
  pipe = make_pipeline(vectorizer, clf)
  pred = clf.predict(vectorizer.transform([text])[0])

  explainer = LimeTextExplainer(class_names=[0, 1])
  exp = explainer.explain_instance(text, pipe.predict_proba)

  result = exp.as_list()
  toxic_list = [x[0] for x in result if x[1] >= 0.1]

  filtered = text
  for toxic in toxic_list:
    filtered = filtered.replace(toxic, "*" * len(toxic))

  print("=====================")
  print("Original Text: \n{}\n\nFiltered: \n{}".format(text, filtered))
  print("=====================")

In [ ]:
filter_toxic("나는 존나 바보입니다.")